In [1]:
import pandas as pd
import os
import protfasta
import glob
from Bio.Seq import Seq
import pickle

First, downloading the transcripts

In [2]:
# ! mkdir /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca

In [3]:
all_TFs = pd.read_csv("../soto_analysis/outputs/all_TFs_table_proteins.txt", sep = "\t", index_col = 0)
all_TFs

,1,2,uniprotID,ENSG,ENST,DBD_coords,AD_coords,RD_coords,Bif_coords,length
0,NaN,NaN,A0A087WUV0,NaN,ENST00000425953,"221-243,249-271,277-299,305-327,333-355,361-38...",NaN,NaN,NaN,1
1,NaN,NaN,A0AVK6,NaN,ENST00000250024,"114-182,262-347",NaN,NaN,NaN,1
2,NaN,NaN,A1YPR0,NaN,ENST00000535628,"364-386,392-414,420-442,448-469",NaN,NaN,NaN,1
3,NaN,NaN,A2RRD8,NaN,ENST00000391781,"161-183,189-211,217-239,245-267,273-295,301-32...",NaN,NaN,NaN,1
4,NaN,NaN,A2RU54,NaN,ENST00000339992,150-206,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
1585,NaN,NaN,Q9Y692,NaN,ENST00000294409,89-165,422-573,NaN,NaN,1
1586,NaN,NaN,Q9Y6Q3,NaN,ENST00000374227,"293-315,321-343,377-399,405-427,433-455,461-48...",NaN,NaN,NaN,1
1587,NaN,NaN,Q9Y6Q9,NaN,ENST00000371998,31-83,621-1424,NaN,NaN,1
1588,NaN,NaN,Q9Y6X0,NaN,ENST00000649279,"583-596,1015-1027,1450-1462",NaN,NaN,NaN,1


In [4]:
SFARI_TFs = pd.read_csv("../data/SFARI_TFs_with_ENST_corrected.csv")
SFARI_TFs

,Unnamed: 0,uniprotID,ENST
0,0,Q9H2P0,ENST00000349014.8
1,1,Q5TGY3,ENST00000247087.10
2,2,Q96QS3,ENST00000379044.5
3,3,Q96JM3,ENST00000361283.4
4,4,Q96RK0,ENST00000575354.6
...,...,...,...
117,117,Q8NEK5,ENST00000366197.9
118,118,Q9BR84,ENST00000393883.6
119,119,Q68DY1,ENST00000601440.6
120,120,Q9Y462,ENST00000276123.7


In [5]:
SFARI_TFs_Table = all_TFs[all_TFs["uniprotID"].isin(SFARI_TFs["uniprotID"])]
SFARI_TFs_Table

,1,2,uniprotID,ENSG,ENST,DBD_coords,AD_coords,RD_coords,Bif_coords,length
47,NaN,NaN,O14529,NaN,ENST00000261726,"549-627,892-967,1043-1120,1169-1225",NaN,NaN,NaN,1
98,NaN,NaN,O95096,NaN,ENST00000377142,129-185,220-273,NaN,NaN,1
102,NaN,NaN,O95365,NaN,ENST00000322357,"382-404,410-432,438-460,466-487",NaN,NaN,NaN,1
135,NaN,NaN,P10589,NaN,ENST00000327111,84-153,NaN,NaN,NaN,1
138,NaN,NaN,P12755,NaN,ENST00000378536,1-728,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
1556,NaN,NaN,Q9UPW6,NaN,ENST00000260926,"355-434,479-557,615-672",NaN,NaN,NaN,1
1563,NaN,NaN,Q9Y2K7,NaN,ENST00000529006,563-609,NaN,NaN,NaN,1
1571,NaN,NaN,Q9Y458,NaN,ENST00000373294,94-283,402-481,NaN,NaN,1
1572,NaN,NaN,Q9Y462,NaN,ENST00000276123,"383-405,414-436,476-499,505-527,533-556,562-58...",NaN,NaN,NaN,1


In [6]:
# Use uniprot to get gene names 
SFARI_TFs_Table[["uniprotID"]].drop_duplicates().to_csv("../data/SFARI_TF_uniprotIDs.csv", index = None, header = None)

In [7]:
gene_names = pd.read_csv("../data/SFARI_TF_gene_names.tsv", sep = "\t")
gene_names["Gene Names"] = gene_names["Gene Names"].str.split(" ").str[0]
gene_names = gene_names[["From", "Gene Names"]]
gene_names = gene_names.rename(columns = {"From" : "uniprotID", "Gene Names" : "gene"})
gene_names

,uniprotID,gene
0,O14770,MEIS2
1,O75840,KLF7
2,O94983,CAMTA2
3,O95096,NKX2-2
4,P10275,AR
5,P10827,THRA
6,P11308,ERG
7,P11473,VDR
8,P15884,TCF4
9,P19532,TFE3


In [8]:
input_tbl = pd.merge(SFARI_TFs_Table[["uniprotID", "ENST"]], gene_names, on = "uniprotID", how = "left")
input_tbl

,uniprotID,ENST,gene
0,O14529,ENST00000261726,NaN
1,O95096,ENST00000377142,NKX2-2
2,O95365,ENST00000322357,NaN
3,P10589,ENST00000327111,NaN
4,P12755,ENST00000378536,NaN
...,...,...,...
117,Q9UPW6,ENST00000260926,NaN
118,Q9Y2K7,ENST00000529006,NaN
119,Q9Y458,ENST00000373294,TBX22
120,Q9Y462,ENST00000276123,NaN


In [12]:
# Clicked on links to download, then moved

for ENST, gene in zip(input_tbl["ENST"], input_tbl["gene"]):
    # print(ENST,gene)
    # command = "wget --no-check-certificate https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/" + ENST + "." + gene + ".fasta.gz -P /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca"
    # os.system(command)

    print("https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/" + ENST + "." + gene + ".fasta.gz")

https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000561208.MEIS2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000309446.KLF7.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000348066.CAMTA2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000377142.NKX2-2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000374690.AR.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000264637.THRA.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000288319.ERG.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000395324.VDR.fasta.gz
https://genome.senckenberg.de/download/TOGA/hum

In [62]:
# Focusing on smallest file, YY1 to test
files = glob.glob("../data/zoonomia_toga_mca/ENST00000239243.MSX2.fasta.gz")
files

['../data/zoonomia_toga_mca/ENST00000239243.MSX2.fasta.gz']

In [63]:
#Dna transcripts for ref seq
human_dna_transcripts = pickle.load(open('../soto_analysis/raw_files/dna_transcripts.dat', 'rb'))

In [65]:
for file in files:
    #Uncompress - run once
    os.system("gunzip " + file)

    # Read in as dataframe
    df = protfasta.read_fasta(file[:-3], invalid_sequence_action = 'convert')
    df = pd.DataFrame({"id" : df.keys(), "nt_seq" : df.values()})
    df["prot_seq"] = [str(Seq(_).translate()) for _ in df["nt_seq"]]


    ENST = file.split("/")[-1].split(".")[0]
    gene =  file.split("/")[-1].split(".")[1]
    
    prot_dict = dict(zip(df["id"], df["prot_seq"]))
    prot_dict["human"] = str(Seq(human_dna_transcripts[ENST]).translate())
    
    protfasta.write_fasta(prot_dict, "../data/zoonomia_toga_mca/prot_fastas/" + gene)

In [56]:
# Everything seems very conserved